In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import os
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('C:\\Users\\HP\\Documents\\KESHTECH\\DSML\\Projects\\Diabetes_Prediction\\Diabetes_Classification\\Datasets\\diabetes_binary_5050split_health_indicators_BRFSS2015.csv')

df.head()

,Diabetes_binary,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,Fruits,...,AnyHealthcare,NoDocbcCost,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,Income
0,0.0,1.0,0.0,1.0,26.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,3.0,5.0,30.0,0.0,1.0,4.0,6.0,8.0
1,0.0,1.0,1.0,1.0,26.0,1.0,1.0,0.0,0.0,1.0,...,1.0,0.0,3.0,0.0,0.0,0.0,1.0,12.0,6.0,8.0
2,0.0,0.0,0.0,1.0,26.0,0.0,0.0,0.0,1.0,1.0,...,1.0,0.0,1.0,0.0,10.0,0.0,1.0,13.0,6.0,8.0
3,0.0,1.0,1.0,1.0,28.0,1.0,0.0,0.0,1.0,1.0,...,1.0,0.0,3.0,0.0,3.0,0.0,1.0,11.0,6.0,8.0
4,0.0,0.0,0.0,1.0,29.0,1.0,0.0,0.0,1.0,1.0,...,1.0,0.0,2.0,0.0,0.0,0.0,0.0,8.0,5.0,8.0


In [3]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold, RepeatedStratifiedKFold, RandomizedSearchCV, cross_validate, GridSearchCV
from sklearn.metrics import classification_report

In [4]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import f1_score, recall_score, precision_score

In [5]:
X = df.drop(columns="Diabetes_binary")
y = df["Diabetes_binary"]

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, stratify = y, random_state=42)

# check shapes of split
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(56553, 21)
(14139, 21)
(56553,)
(14139,)


In [7]:
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import MinMaxScaler

def get_pipe(estimator):

    """
    Data passed to this pipeline would be scaled by MinMaxScaler before the classifier
    """
    return Pipeline([('scaler', MinMaxScaler()), ('estimator', estimator)])

#### Random Forest Classifier

In [8]:
pipe_rfc = make_pipeline(
    MinMaxScaler(),
    RandomForestClassifier(
        n_estimators=300,
        criterion='entropy',
        max_depth=15,
        min_samples_split=8,
        min_samples_leaf=3,
        max_features='log2',
        bootstrap=False,
        oob_score=False,
        ccp_alpha=0.005
    )
)
pipe_rfc.fit(X_train, y_train)

Pipeline(steps=[('minmaxscaler', MinMaxScaler()),
                ('randomforestclassifier',
                 RandomForestClassifier(bootstrap=False, ccp_alpha=0.005,
                                        criterion='entropy', max_depth=15,
                                        max_features='log2', min_samples_leaf=3,
                                        min_samples_split=8,
                                        n_estimators=300))])

In [13]:
# Make predictions
y_train_pred = pipe_rfc.predict(X_train)
y_test_pred = pipe_rfc.predict(X_test)

# Training set performance
rfc_train_accuracy = accuracy_score(y_train, y_train_pred) # Calculate Accuracy
rfc_train_mcc = matthews_corrcoef(y_train, y_train_pred) # Calculate MCC
rfc_train_f1 = f1_score(y_train, y_train_pred, average='weighted') # Calculate F1-score
rfc_train_precision = precision_score(y_train, y_train_pred)
rfc_train_recall = recall_score(y_train, y_train_pred)

# Test set performance
rfc_test_accuracy = accuracy_score(y_test, y_test_pred) # Calculate Accuracy
rfc_test_mcc = matthews_corrcoef(y_test, y_test_pred) # Calculate MCC
rfc_test_f1 = f1_score(y_test, y_test_pred, average='weighted') # Calculate F1-score
rfc_test_precision = precision_score(y_test, y_test_pred)
rfc_test_recall = recall_score(y_test, y_test_pred)

print('Model performance for Training set')
print('- Accuracy: %s' % rfc_train_accuracy)
print('- MCC: %s' % rfc_train_mcc)
print('- F1 score: %s' % rfc_train_f1)
print('- Precision: %s' % rfc_train_precision)
print('- Recall: %s' % rfc_train_recall)
print(classification_report(y_train, y_train_pred))
      
print("=========================================================")

print('Model performance for Test set')
print('- Accuracy: %s' % rfc_test_accuracy)
print('- MCC: %s' % rfc_test_mcc)
print('- F1 score: %s' % rfc_test_f1)
print('- Precision: %s' % rfc_test_precision)
print('- Recall: %s' % rfc_test_recall)
print(classification_report(y_test, y_test_pred))

Model performance for Training set
- Accuracy: 0.740155252594911
- MCC: 0.48192758426774407
- F1 score: 0.7397189164657166
- Precision: 0.7219861401673641
- Recall: 0.781094175478304
              precision    recall  f1-score   support

         0.0       0.76      0.70      0.73     28276
         1.0       0.72      0.78      0.75     28277

    accuracy                           0.74     56553
   macro avg       0.74      0.74      0.74     56553
weighted avg       0.74      0.74      0.74     56553

Model performance for Test set
- Accuracy: 0.7351297828700757
- MCC: 0.47201743010146285
- F1 score: 0.7346382863749998
- Precision: 0.7164626204740818
- Recall: 0.778186447870986
              precision    recall  f1-score   support

         0.0       0.76      0.69      0.72      7070
         1.0       0.72      0.78      0.75      7069

    accuracy                           0.74     14139
   macro avg       0.74      0.74      0.73     14139
weighted avg       0.74      0.74     

#### SVM Classifier

In [14]:
pipe_svmp = make_pipeline(MinMaxScaler(), SVC(C = 4, degree = 2, gamma = 'scale', kernel = 'poly'))
pipe_svmp.fit(X_train, y_train)

Pipeline(steps=[('minmaxscaler', MinMaxScaler()),
                ('svc', SVC(C=4, degree=2, kernel='poly'))])

In [16]:
# Make predictions
y_train_pred = pipe_svmp.predict(X_train)
y_test_pred = pipe_svmp.predict(X_test)

# Training set performance
svmp_train_accuracy = accuracy_score(y_train, y_train_pred) # Calculate Accuracy
svmp_train_mcc = matthews_corrcoef(y_train, y_train_pred) # Calculate MCC
svmp_train_f1 = f1_score(y_train, y_train_pred, average='weighted') # Calculate F1-score
svmp_train_precision = precision_score(y_train, y_train_pred)
svmp_train_recall = recall_score(y_train, y_train_pred)

# Test set performance
svmp_test_accuracy = accuracy_score(y_test, y_test_pred) # Calculate Accuracy
svmp_test_mcc = matthews_corrcoef(y_test, y_test_pred) # Calculate MCC
svmp_test_f1 = f1_score(y_test, y_test_pred, average='weighted') # Calculate F1-score
svmp_test_precision = precision_score(y_test, y_test_pred)
svmp_test_recall = recall_score(y_test, y_test_pred)


print('Model performance for Training set')
print('- Accuracy: %s' % svmp_train_accuracy)
print('- MCC: %s' % svmp_train_mcc)
print('- F1 score: %s' % svmp_train_f1)
print('- Precision: %s' % svmp_train_precision)
print('- Recall: %s' % svmp_train_recall)
print(classification_report(y_train, y_train_pred))
print('===================================================')
print('Model performance for Test set')
print('- Accuracy: %s' % svmp_test_accuracy)
print('- MCC: %s' % svmp_test_mcc)
print('- F1 score: %s' % svmp_test_f1)
print('- Precision: %s' % svmp_test_precision)
print('- Recall: %s' % svmp_test_recall)
print(classification_report(y_test, y_test_pred))

Model performance for Training set
- Accuracy: 0.7529220377345145
- MCC: 0.5113505658369074
- F1 score: 0.7515911171098004
- Precision: 0.7206317867719645
- Recall: 0.8261130954485978
              precision    recall  f1-score   support

         0.0       0.80      0.68      0.73     28276
         1.0       0.72      0.83      0.77     28277

    accuracy                           0.75     56553
   macro avg       0.76      0.75      0.75     56553
weighted avg       0.76      0.75      0.75     56553

Model performance for Test set
- Accuracy: 0.7492043284532145
- MCC: 0.5034760156741624
- F1 score: 0.7479447603364326
- Precision: 0.7183046226298179
- Recall: 0.8199179516197482
              precision    recall  f1-score   support

         0.0       0.79      0.68      0.73      7070
         1.0       0.72      0.82      0.77      7069

    accuracy                           0.75     14139
   macro avg       0.75      0.75      0.75     14139
weighted avg       0.75      0.75    

#### XGBoost Classifier

In [17]:
pipe_xgb = make_pipeline(
    MinMaxScaler(),
    XGBClassifier(
        booster='gbtree',
        learning_rate=0.1,
        n_estimators=200,
        max_depth=5,
        min_child_weight=2,
        gamma=0.1,
        subsample=0.9,
        colsample_bytree=0.8,
        reg_alpha=0.01,
        reg_lambda=1.5
    )
)
pipe_xgb.fit(X_train, y_train)

Pipeline(steps=[('minmaxscaler', MinMaxScaler()),
                ('xgbclassifier',
                 XGBClassifier(base_score=None, booster='gbtree',
                               callbacks=None, colsample_bylevel=None,
                               colsample_bynode=None, colsample_bytree=0.8,
                               early_stopping_rounds=None,
                               enable_categorical=False, eval_metric=None,
                               feature_types=None, gamma=0.1, gpu_id=None,
                               grow_policy=None, importance_type=None,
                               interaction_constraints=None, learning_rate=0.1,
                               max_bin=None, max_cat_threshold=None,
                               max_cat_to_onehot=None, max_delta_step=None,
                               max_depth=5, max_leaves=None, min_child_weight=2,
                               missing=nan, monotone_constraints=None,
                               n_estimators=200, n_jobs=None,
                               num_parallel_tree=None, predictor=None,
                               random_state=None, ...))])

In [19]:
# Make predictions
y_train_pred = pipe_xgb.predict(X_train)
y_test_pred = pipe_xgb.predict(X_test)

# Training set performance
xgb_train_accuracy = accuracy_score(y_train, y_train_pred) # Calculate Accuracy
xgb_train_mcc = matthews_corrcoef(y_train, y_train_pred) # Calculate MCC
xgb_train_f1 = f1_score(y_train, y_train_pred, average='weighted') # Calculate F1-score
xgb_train_precision = precision_score(y_train, y_train_pred)
xgb_train_recall = recall_score(y_train, y_train_pred)

# Test set performance
xgb_test_accuracy = accuracy_score(y_test, y_test_pred) # Calculate Accuracy
xgb_test_mcc = matthews_corrcoef(y_test, y_test_pred) # Calculate MCC
xgb_test_f1 = f1_score(y_test, y_test_pred, average='weighted') # Calculate F1-score
xgb_test_precision = precision_score(y_test, y_test_pred)
xgb_test_recall = recall_score(y_test, y_test_pred)


print('Model performance for Training set')
print('- Accuracy: %s' % xgb_train_accuracy)
print('- MCC: %s' % xgb_train_mcc)
print('- F1 score: %s' % xgb_train_f1)
print('- Precision: %s' % xgb_train_precision)
print('- Recall: %s' % xgb_train_recall)
print(classification_report(y_train, y_train_pred))
print('===================================================')
print('Model performance for Test set')
print('- Accuracy: %s' % xgb_test_accuracy)
print('- MCC: %s' % xgb_test_mcc)
print('- F1 score: %s' % xgb_test_f1)
print('- Precision: %s' % xgb_test_precision)
print('- Recall: %s' % xgb_test_recall)
print(classification_report(y_test, y_test_pred))

Model performance for Training set
- Accuracy: 0.7687832652556009
- MCC: 0.5397999708953646
- F1 score: 0.7683045892210911
- Precision: 0.7463934904528804
- Recall: 0.814230646815433
              precision    recall  f1-score   support

         0.0       0.80      0.72      0.76     28276
         1.0       0.75      0.81      0.78     28277

    accuracy                           0.77     56553
   macro avg       0.77      0.77      0.77     56553
weighted avg       0.77      0.77      0.77     56553

Model performance for Test set
- Accuracy: 0.7540844472734989
- MCC: 0.5105184751918528
- F1 score: 0.7535201890993468
- Precision: 0.7318616059901885
- Recall: 0.8019521855990946
              precision    recall  f1-score   support

         0.0       0.78      0.71      0.74      7070
         1.0       0.73      0.80      0.77      7069

    accuracy                           0.75     14139
   macro avg       0.76      0.75      0.75     14139
weighted avg       0.76      0.75     

#### Stacked Model

In [20]:
# Define estimators
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression

estimator_list = [
    ('pipe_svmp',pipe_svmp),
    ('pipe_rfc',pipe_rfc),
    ('pipe_xgb',pipe_xgb)]

pipe_lr = make_pipeline(
    MinMaxScaler(),
    LogisticRegression()
)

# Build stack model
stack_model = StackingClassifier(
    estimators=estimator_list, final_estimator=pipe_lr
)

# Train stacked model
stack_model.fit(X_train, y_train)

# Make predictions
y_train_pred = stack_model.predict(X_train)
y_test_pred = stack_model.predict(X_test)

# Training set model performance
stack_model_train_accuracy = accuracy_score(y_train, y_train_pred) # Calculate Accuracy
stack_model_train_mcc = matthews_corrcoef(y_train, y_train_pred) # Calculate MCC
stack_model_train_f1 = f1_score(y_train, y_train_pred, average='weighted') # Calculate F1-score
stack_model_train_precision = precision_score(y_train, y_train_pred)
stack_model_train_recall = recall_score(y_train, y_train_pred)

# Test set model performance
stack_model_test_accuracy = accuracy_score(y_test, y_test_pred) # Calculate Accuracy
stack_model_test_mcc = matthews_corrcoef(y_test, y_test_pred) # Calculate MCC
stack_model_test_f1 = f1_score(y_test, y_test_pred, average='weighted') # Calculate F1-score
stack_model_test_precision = precision_score(y_test, y_test_pred)
stack_model_test_recall = recall_score(y_test, y_test_pred)


print('Model performance for Training set')
print('- Accuracy: %s' % stack_model_train_accuracy)
print('- MCC: %s' % stack_model_train_mcc)
print('- F1 score: %s' % stack_model_train_f1)
print('----------------------------------')
print('Model performance for Test set')
print('- Accuracy: %s' % stack_model_test_accuracy)
print('- MCC: %s' % stack_model_test_mcc)
print('- F1 score: %s' % stack_model_test_f1)

Model performance for Training set
- Accuracy: 0.7634785068873446
- MCC: 0.5290279886735786
- F1 score: 0.7630152059012689
----------------------------------
Model performance for Test set
- Accuracy: 0.7521041092014994
- MCC: 0.5061806176746867
- F1 score: 0.7516226586759192


#### Results

In [26]:
acc_train_list = {'svmp': svmp_train_accuracy,
'xgb': xgb_train_accuracy,
'rfc': rfc_train_accuracy,
'stack': stack_model_train_accuracy}

mcc_train_list = {'svmp': svmp_train_mcc,
'xgb': xgb_train_mcc,
'rfc': rfc_train_mcc,
'stack': stack_model_train_mcc}

f1_train_list = {'svmp': svmp_train_f1,
'xgb': xgb_train_f1,
'rfc': rfc_train_f1,
'stack': stack_model_train_f1}

precision_train_list = {'svmp': svmp_train_precision,
'xgb': xgb_train_precision,
'rfc': rfc_train_precision,
'stack': stack_model_train_precision}

recall_train_list = {'svmp': svmp_train_recall,
'xgb': xgb_train_recall,
'rfc': rfc_train_recall,
'stack': stack_model_train_recall}

In [27]:
acc_test_list = {'svmp': svmp_test_accuracy,
'xgb': xgb_test_accuracy,
'rfc': rfc_test_accuracy,
'stack': stack_model_test_accuracy}

mcc_test_list = {'svmp': svmp_test_mcc,
'xgb': xgb_test_mcc,
'rfc': rfc_test_mcc,
'stack': stack_model_test_mcc}

f1_test_list = {'svmp': svmp_test_f1,
'xgb': xgb_test_f1,
'rfc': rfc_test_f1,
'stack': stack_model_test_f1}

precision_test_list = {'svmp': svmp_test_precision,
'xgb': xgb_test_precision,
'rfc': rfc_test_precision,
'stack': stack_model_test_precision}

recall_test_list = {'svmp': svmp_test_recall,
'xgb': xgb_test_recall,
'rfc': rfc_test_recall,
'stack': stack_model_test_recall}

In [28]:
acc_df = pd.DataFrame.from_dict(acc_train_list, orient='index', columns=['Accuracy'])
mcc_df = pd.DataFrame.from_dict(mcc_train_list, orient='index', columns=['MCC'])
f1_df = pd.DataFrame.from_dict(f1_train_list, orient='index', columns=['F1'])
precision_df = pd.DataFrame.from_dict(precision_train_list, orient='index', columns=['Precision'])
recall_df = pd.DataFrame.from_dict(recall_train_list, orient='index', columns=['Recall'])
df_metrics_train = pd.concat([acc_df, mcc_df, f1_df, precision_df, recall_df], axis=1)
df_metrics_train

,Accuracy,MCC,F1,Precision,Recall
svmp,0.752922,0.511351,0.751591,0.720632,0.826113
xgb,0.768783,0.539800,0.768305,0.746393,0.814231
rfc,0.740155,0.481928,0.739719,0.721986,0.781094
stack,0.763479,0.529028,0.763015,0.742080,0.807688


In [29]:
acc_test_df = pd.DataFrame.from_dict(acc_test_list, orient='index', columns=['Accuracy'])
mcc_test_df = pd.DataFrame.from_dict(mcc_test_list, orient='index', columns=['MCC'])
f1_test_df = pd.DataFrame.from_dict(f1_test_list, orient='index', columns=['F1'])
precision_test_df = pd.DataFrame.from_dict(precision_test_list, orient='index', columns=['Precision'])
recall_test_df = pd.DataFrame.from_dict(recall_test_list, orient='index', columns=['Recall'])
df_metrics_test = pd.concat([acc_test_df, mcc_test_df, f1_test_df, precision_test_df, recall_test_df], axis=1)
df_metrics_test

,Accuracy,MCC,F1,Precision,Recall
svmp,0.749204,0.503476,0.747945,0.718305,0.819918
xgb,0.754084,0.510518,0.753520,0.731862,0.801952
rfc,0.735130,0.472017,0.734638,0.716463,0.778186
stack,0.752104,0.506181,0.751623,0.731669,0.796152
